In [64]:
import pickle
import matplotlib.pyplot as plt
import torch
import numpy as np
from src.Dataset_loader import unpickle, CustomImageDataset

def show(img):
    img = img/255
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)), interpolation='nearest')
    plt.show()


In [67]:
CIFAR10_train_dataset = CustomImageDataset()
CIFAR10_test_dataset = CustomImageDataset()

In [68]:
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_1')
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_2')
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_3')
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_4')
CIFAR10_train_dataset.load_batch('data/cifar-10-batches-py/data_batch_5')

In [69]:
CIFAR10_test_dataset.load_batch('data/cifar-10-batches-py/test_batch')

In [61]:
image, label = next(iter(Dataset))